# Neural Networks
## _MNIST Dataset_

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [20]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import time
import os

In [21]:
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script src="//cdnjs.cloudflare.com/ajax/libs/polymer/0.3.3/platform.js"></script>
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [22]:
print(tf.__version__)

1.10.0


In [23]:
from datetime import datetime

def log_dir(prefix=""):
    now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
    root_logdir = "tf_logs"
    if prefix:
        prefix += "-"
        name = prefix + "run-" + now
    return "{}/run-{}/".format(root_logdir,now)

In [24]:
# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
#%matplotlib inline
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "C:\\Users\\Alex\\Documents\\Practical Machine Learning\\group3\\6"
CHAPTER_ID = "Artificial Neural Networks"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

In [25]:
#Gathering the MNIST Dataset
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

###  Neural Network Construction - 2 Hidden Layers, 300 nodes in Layer 1, 100 nodes in Layer 2, 20 Steps

In [26]:
#Network Parameters
n_inputs = 784    #28*28 MNIST
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [27]:
reset_graph()
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

In [28]:
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1",
                              activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2",
                              activation=tf.nn.relu)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")
    y_proba = tf.nn.softmax(logits)
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    loss_summary = tf.summary.scalar('log_loss', loss)

learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    accuracy_summary = tf.summary.scalar('accuracy', accuracy)
    
init = tf.global_variables_initializer()
saver = tf.train.Saver() 

In [29]:
n_epochs = 20
n_batches = 50

In [32]:
#shuffling the batch
def shuffle_batch(X, y, n_batches):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // n_batches
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch    #Generator function

In [33]:
logdir = log_dir("mnist_dnn")

In [34]:
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

### Execution Phase

In [35]:
start_time = time.time()
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, n_batches):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val, loss_val, accuracy_summary_str, loss_summary_str = sess.run([accuracy, 
                                                                                   loss, accuracy_summary, 
                                                                                   loss_summary], 
                                                                                  feed_dict={X: X_valid, y: y_valid})
        print("Epoch:", epoch,
                  "\tValidation accuracy: {:.3f}%".format(accuracy_val * 100),
                  "\tLoss: {:.5f}".format(loss_val))
        save_path = saver.save(sess, "./my_model_final.ckpt")
end_time = time.time()
total_time = end_time - start_time
print ("Running Time: {0:.0f} Seconds".format(total_time))

Epoch: 0 	Validation accuracy: 90.240% 	Loss: 0.35380
Epoch: 1 	Validation accuracy: 92.540% 	Loss: 0.27323
Epoch: 2 	Validation accuracy: 93.740% 	Loss: 0.23673
Epoch: 3 	Validation accuracy: 94.180% 	Loss: 0.21018
Epoch: 4 	Validation accuracy: 94.720% 	Loss: 0.19066
Epoch: 5 	Validation accuracy: 95.140% 	Loss: 0.17919
Epoch: 6 	Validation accuracy: 95.460% 	Loss: 0.16222
Epoch: 7 	Validation accuracy: 96.100% 	Loss: 0.15050
Epoch: 8 	Validation accuracy: 96.220% 	Loss: 0.14200
Epoch: 9 	Validation accuracy: 96.480% 	Loss: 0.13554
Epoch: 10 	Validation accuracy: 96.520% 	Loss: 0.12783
Epoch: 11 	Validation accuracy: 96.680% 	Loss: 0.12106
Epoch: 12 	Validation accuracy: 96.840% 	Loss: 0.12012
Epoch: 13 	Validation accuracy: 97.040% 	Loss: 0.11289
Epoch: 14 	Validation accuracy: 96.960% 	Loss: 0.11039
Epoch: 15 	Validation accuracy: 97.180% 	Loss: 0.10326
Epoch: 16 	Validation accuracy: 97.280% 	Loss: 0.10115
Epoch: 17 	Validation accuracy: 97.280% 	Loss: 0.09962
Epoch: 18 	Validatio

In [36]:
file_writer.close()

In [37]:
with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt") # or better, use save_path
    X_new_scaled = X_test[30:50]
    Z = logits.eval(feed_dict={X: X_new_scaled})
    y_pred = np.argmax(Z, axis=1)

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt


In [38]:
print("Predicted:", y_pred)
print("Actual:   ", y_test[30:50])

Predicted: [3 1 3 4 7 2 7 1 2 1 1 7 4 2 3 5 1 2 4 4]
Actual:    [3 1 3 4 7 2 7 1 2 1 1 7 4 2 3 5 1 2 4 4]


In [39]:
show_graph(tf.get_default_graph())

### Neural Network - 3 Hidden Layers, 250 nodes in Layer 1, 150 nodes in Layer 2, 100 nodes in Layer 3, 200 Steps

In [61]:
#Network Parameters
n_inputs = 784    #28*28 MNIST
n_hidden1 = 250
n_hidden2 = 150
n_hidden3 = 100
n_outputs = 10

In [62]:
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [63]:
reset_graph()
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

In [64]:
with tf.name_scope("dnn2"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1",
                              activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2",
                              activation=tf.nn.relu)
    hidden3 = tf.layers.dense(hidden2, n_hidden3, name="hidden3",
                              activation=tf.nn.relu)
    logits = tf.layers.dense(hidden3, n_outputs, name="outputs")
    y_proba = tf.nn.softmax(logits)
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    loss_summary = tf.summary.scalar('log_loss', loss)

learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    accuracy_summary = tf.summary.scalar('accuracy', accuracy)
    
init = tf.global_variables_initializer()
saver = tf.train.Saver() 

In [65]:
n_epochs = 100
n_batches = 50

start_time = time.time()
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, n_batches):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val, loss_val, accuracy_summary_str, loss_summary_str = sess.run([accuracy, 
                                                                                   loss, accuracy_summary, 
                                                                                   loss_summary], 
                                                                                  feed_dict={X: X_valid, y: y_valid})
        file_writer.add_summary(accuracy_summary_str, epoch)
        file_writer.add_summary(loss_summary_str, epoch)
        print("Epoch:", epoch,
                  "\tValidation accuracy: {:.3f}%".format(accuracy_val * 100),
                  "\tLoss: {:.5f}".format(loss_val))
        save_path = saver.save(sess, "./my_model_final.ckpt")
end_time = time.time()
total_time = end_time - start_time
print ("Running Time: {0:.0f} Seconds".format(total_time))

Epoch: 0 	Validation accuracy: 90.540% 	Loss: 0.34337
Epoch: 1 	Validation accuracy: 92.560% 	Loss: 0.26469
Epoch: 2 	Validation accuracy: 93.660% 	Loss: 0.22160
Epoch: 3 	Validation accuracy: 94.340% 	Loss: 0.19570
Epoch: 4 	Validation accuracy: 95.220% 	Loss: 0.17440
Epoch: 5 	Validation accuracy: 95.000% 	Loss: 0.16905
Epoch: 6 	Validation accuracy: 96.100% 	Loss: 0.14489
Epoch: 7 	Validation accuracy: 96.360% 	Loss: 0.13255
Epoch: 8 	Validation accuracy: 96.440% 	Loss: 0.12779
Epoch: 9 	Validation accuracy: 96.480% 	Loss: 0.12141
Epoch: 10 	Validation accuracy: 96.740% 	Loss: 0.11369
Epoch: 11 	Validation accuracy: 96.880% 	Loss: 0.10602
Epoch: 12 	Validation accuracy: 96.660% 	Loss: 0.10997
Epoch: 13 	Validation accuracy: 96.940% 	Loss: 0.10293
Epoch: 14 	Validation accuracy: 97.080% 	Loss: 0.09730
Epoch: 15 	Validation accuracy: 97.280% 	Loss: 0.09395
Epoch: 16 	Validation accuracy: 97.320% 	Loss: 0.09251
Epoch: 17 	Validation accuracy: 97.420% 	Loss: 0.08837
Epoch: 18 	Validatio

In [66]:
file_writer.close()

In [67]:
with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt") # or better, use save_path
    X_new_scaled = X_test[100:130]
    Z = logits.eval(feed_dict={X: X_new_scaled})
    y_pred = np.argmax(Z, axis=1)

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt


In [68]:
print("Predicted:", y_pred)
print("Actual:   ", y_test[100:130])

Predicted: [6 0 5 4 9 9 2 1 9 4 8 7 3 9 7 9 4 4 9 2 5 4 7 6 7 9 0 5 8 5]
Actual:    [6 0 5 4 9 9 2 1 9 4 8 7 3 9 7 4 4 4 9 2 5 4 7 6 7 9 0 5 8 5]


In [69]:
show_graph(tf.get_default_graph())